In [3]:
#-*- coding: cp949 -*-
#-*- coding: utf-8 -*- 
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim

from sklearn.datasets import load_iris

iris= load_iris()
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

ModuleNotFoundError: No module named 'pandas'

In [130]:
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [131]:
data_set = data.iloc[:,[0,1,2,3]].values.tolist()
data_set[0]

[5.1, 3.5, 1.4, 0.2]

In [132]:
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self, df):
    #데이터셋의 전처리를 해주는 부분
    
    #sepal lenght,sepal width, petal length, petal width
        self.x_data = df.iloc[:,[0,1,2]].values.tolist()
        self.petal_width = df.iloc[:, [3]].values.tolist()
        self.target = df.iloc[:, [4]].values.tolist()

    def __len__(self):
    #데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
        return len(self.x_data)

    def __getitem__(self, idx): 
    #데이터셋에서 특정 1개의 샘플을 가져오는 함수
        x = torch.tensor(self.x_data[idx], dtype = torch.float32)
        pw = torch.tensor(self.petal_width[idx], dtype = torch.float32)
        target = torch.tensor(self.target[idx], dtype = torch.float32)
        return x,pw,target
        

In [133]:
#데이터로더 확인
import torch
import torchvision.transforms as transforms

dataset = CustomDataset(data)

#batch_size 만큼 묶어서 데이터 (batch_size가 커지면 속도가 빨리짐, gpu 사용 증가)
dataLoader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

In [134]:
dataiter = iter(dataLoader)
feature, width, target = dataiter.next()
print(feature)
print(width)
print(target)

tensor([[4.9000, 3.0000, 1.4000],
        [6.0000, 2.9000, 4.5000],
        [6.4000, 2.8000, 5.6000],
        [4.9000, 3.6000, 1.4000]])
tensor([[0.2000],
        [1.5000],
        [2.1000],
        [0.1000]])
tensor([[0.],
        [1.],
        [2.],
        [0.]])


In [135]:
#test, train 비율
split_ratio = 0.7
split_index = int(len(data) * split_ratio)

#train, test셋 분류
#데이터가 불균형하게 분포되어 있을 경우 사용 x
train_dataset = CustomDataset(data[:split_index])
test_dataset = CustomDataset(data[split_index:len(data)])

#데이터 로더
train_dataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

print(len(train_dataLoader))
print(len(test_dataLoader))

27
12


In [136]:
import torch.nn as nn
from torch import optim

class TabularClassificaation(nn.Module):
    def __init__(self, num_features):
        super(TabularClassificaation, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 512)
        self.layer_2 = nn.Linear(512, 1024)                                                                                                                                    
        self.layer_3 = nn.Linear(1024, 512)
        self.layer_out1 = nn.Linear(512, 3)
        self.layer_out2 = nn.Linear(512, 1)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x1 = self.softmax(self.layer_out1(x))
        x2 = self.relu(self.layer_out2(x))
        
        return x1, x2

In [137]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TabularClassificaation(num_features=3).to(device)
num_epochs = 500
loss_func1 = nn.CrossEntropyLoss()
loss_func2 = nn.MSELoss()

opt = optim.Adam(model.parameters(), lr=0.00001)

In [140]:
for epoch in range(num_epochs):
    running_loss1 = 0.0
    running_loss2 = 0.0
    total_loss = 0.0
    
    for data in train_dataLoader:
        features, gt_petal, gt_target = data[0].to(device), data[1].to(device) , data[2].to(device)
        
        opt.zero_grad()
        pred_target, pred_petal = model(features)
        loss1 = loss_func1(pred_target, gt_target.view(-1).long())
        loss2 = loss_func2(pred_petal, gt_petal)
        
        loss = loss1 + loss2
        
        loss.backward()
        
        opt.step()
        
        running_loss1 += loss1.item()
        running_loss2 += loss2.item()
        total_loss += loss.item()
    
    cost1 = running_loss1 / len(train_dataLoader)
    cost2 = running_loss2 / len(train_dataLoader)
    cost = total_loss / len(train_dataLoader)
    
    if epoch % 10 == 0 :
        print('epoch %d, target loss: %.3f, petal width loss: %.3f, total loss: %.3f' %(epoch, cost1, cost2, cost))


epoch 0, target loss: 0.673, petal width loss: 0.019, total loss: 0.692
epoch 10, target loss: 0.627, petal width loss: 0.016, total loss: 0.643
epoch 20, target loss: 0.612, petal width loss: 0.016, total loss: 0.628
epoch 30, target loss: 0.607, petal width loss: 0.014, total loss: 0.621
epoch 40, target loss: 0.604, petal width loss: 0.014, total loss: 0.618
epoch 50, target loss: 0.602, petal width loss: 0.013, total loss: 0.615
epoch 60, target loss: 0.601, petal width loss: 0.014, total loss: 0.615
epoch 70, target loss: 0.600, petal width loss: 0.013, total loss: 0.613
epoch 80, target loss: 0.600, petal width loss: 0.013, total loss: 0.613
epoch 90, target loss: 0.599, petal width loss: 0.014, total loss: 0.613
epoch 100, target loss: 0.599, petal width loss: 0.015, total loss: 0.614
epoch 110, target loss: 0.599, petal width loss: 0.013, total loss: 0.612
epoch 120, target loss: 0.599, petal width loss: 0.013, total loss: 0.611
epoch 130, target loss: 0.599, petal width loss: 

KeyboardInterrupt: 

In [142]:
model(torch.Tensor([[1,1,1],
                    [1,1,1],
                    [1,1,1],
                    [1,1,1]]))

(tensor([[0.4106, 0.5077, 0.0817],
         [0.4106, 0.5077, 0.0817],
         [0.4106, 0.5077, 0.0817],
         [0.4106, 0.5077, 0.0817]], grad_fn=<SoftmaxBackward>),
 tensor([[0.2722],
         [0.2722],
         [0.2722],
         [0.2722]], grad_fn=<ReluBackward0>))